In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ['JAVA HOME']='/usr/lib/jvm/java-8-openjdk-amd64'

openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [ ]:
#create session
conf=SparkConf().set("spark.ui.port", "4050")

In [ ]:
#create context
sc=pyspark.SparkContext(conf=conf)
spark=SparkSession.builder.getOrCreate()

In [ ]:
spark

In [ ]:
import itertools

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import sys
import numpy as np
import time
from itertools import combinations
from pyspark import SparkContext

In [ ]:

# function to find pairs of frequent item combination of all possible size.
def get_item_subset(freq_items,k):
  # simple list to save newly generated pairs
    subset_list = []
    if k == 2:
      # loop through single frequent items to make pairs
        for i,x in enumerate(freq_items):
            for j,y in enumerate(freq_items):
                if j>i:
                    pair = x|y
                  # check if pair is not already exist in list
                    if len(pair) == k and pair not in subset_list:
                        subset_list.append(pair)
    else:
      # to make subset of item greater than size 2
        for i,x in enumerate(freq_items):
            for j,y in enumerate(freq_items):
                if j>i:
                  # now, we have size 2 of subset and we want to make size 3 subset.
                  # first we extract common item between x and y
                    common_item = x.intersection(y)
                    if len(common_item) == k-2:
                        pair = x|y
                        if pair not in subset_list:
                          # make different subset of newly make pair to make sure all of its subset are already 
                          # in frequent list
                            pairs = list(combinations(pair,k-1))
                            c = 0
                            for p in pairs:
                                if c == k-2:
                                    break
                                if common_item.issubset(p) == False:
                                    if set(p) in freq_items:
                                        c = c + 1
                            if c == k-2:
                                subset_list.append(pair)
    return subset_list

def apriori(iterator):
  # this apriori function will run at each partition of dataset
    data = list(iterator)
    # total rows of data in respective parition 
    N=len(data)
    s = len(data) * support/100 
    # support threshold for deciding frequent values
    support_thres = s/numPartitions
    # inrementor for value sets
    k = 2
    l = {}
    freq_items = []
    # to keep final frequent itemsets
    frequent_itemset = []
    # traverse data to find frequent items
    for row in data:
        for item in row:
            if item not in l:
                l[item] = 1
                # maintaing dictionary for each item set support count
            else:
                l[item] = l[item] + 1
    
    for item in l:
      # if all items total occurance in greater than mention threshold. we'll apend to
      # frequent list
        if l[item] >= support_thres:
            freq_items.append(item)
    # save into final frequent itemset list
    frequent_itemset = [(i,1) for i in freq_items]
    freq_items = [{i} for i in freq_items]
    
    while (len(freq_items)>0):
      # now we make pair of frequent items.
        items_set = get_item_subset(freq_items,k)
        # empty freqent items array for new frequent items pair of size 2
        freq_items = []
        count={}
        for item in items_set:
            c = 0
            for basket in data:
              # check how many times generated pair is exist in partition data
                if item.issubset(basket):
                    c = c + 1
                    # occurnace more than threshold, then it's a frequent combination of items
            if c >= support_thres:
                count[tuple(item)] = c
                freq_items.append(item)
          # now, we'll append all frequent items to frequent itemset 
        if len(freq_items)>0:
            frequent_itemset = frequent_itemset + [(tuple(i),1) for i in freq_items]
            # increment value of K for bigger size of item combination
        k = k + 1
    return(frequent_itemset)


In [ ]:
# this function will scan the entire data set and find total number of occurance of each frequent itemset.
def totalCount_itemset(iterator,itemset):
    data = list(iterator)
    item_local = []
    # loop through each frequent item
    for item in itemset:
        c = 0
        if type(item) != tuple:
            item = (item,)
            # loop entire data set
        for row in data:
          # count occurance of each frequnet item set in data base
            flag =  all(value in row  for value in item)
            if flag == True:
                c = c + 1
                # save frequent items with occurance
        item_local.append((item,c))
    
    return(item_local)

In [ ]:
data = sc.textFile('/content/drive/MyDrive/Dataset.csv', minPartitions=2)

In [ ]:
# total number of rows
data.collect()

['age,workclass,education,Education_numOfYears,marital_status,occupation,relationship,race,sex,hous_per_week,native_country,fifty_bool',
 'Young-Age,Private,Incomplete-Education,7,Never-married,Technical-Working-Class, Own-child, Black, Male,Part-Time-Hours, United-States, <=50K.',
 'Middle-Age,Private,HS-Graduates,9,Married,Lowwer-Working-Class, Husband, White, Male,Normal-Hours, United-States, <=50K.',
 'Young-Age,Goverment,Associates,12,Married,Other-Service, Husband, White, Male,Part-Time-Hours, United-States, >50K.',
 'Middle-Age,Private,HS-Graduates,10,Married,Technical-Working-Class, Husband, Black, Male,Part-Time-Hours, United-States, >50K.',
 'Young-Age,Private,Incomplete-Education,6,Never-married,Other-Service, Not-in-family, White, Male,Part-Time-Hours, United-States, <=50K.',
 'Old-Age,Self-Employed,Incomplete-Education,15,Married,Technical-Working-Class, Husband, White, Male,Part-Time-Hours, United-States, >50K.',
 'Young-Age,Private,HS-Graduates,10,Never-married,Other-Ser

In [ ]:
df=spark.read.csv('/content/drive/MyDrive/Dataset.csv',sep=',',inferSchema=True,header=True)

In [ ]:
# data preprocessing
#  Handle the missing values
count=[]

#save the names of columns in cols variable
cols=df.columns
print('Columns in the dataset:',cols)
print('\n')

#counting the null values in each column
print('Number of null values in each column:')

#for loop to count and append the nulll values in columns
for cols in df.columns:
  arr=df.where(df[cols].isNull()).count()
  count.append(arr)
  print(cols," ",arr)

print('Array of missing values: ',count)
print("\n")


Columns in the dataset: ['age', 'workclass', 'education', 'Education_numOfYears', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'hous_per_week', 'native_country', 'fifty_bool']


Number of null values in each column:
age   0
workclass   0
education   0
Education_numOfYears   0
marital_status   0
occupation   0
relationship   0
race   0
sex   0
hous_per_week   0
native_country   0
fifty_bool   0
Array of missing values:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]




In [ ]:
header = data.first()

In [ ]:
print(header)

age,workclass,education,Education_numOfYears,marital_status,occupation,relationship,race,sex,hous_per_week,native_country,fifty_bool


In [ ]:
# we'll remove header row of our data set and split the data at comma
dataset = data.filter(lambda x: x!=header).map(lambda x: x.split(',')).map(lambda x: ((x[0]),(x[1]),(x[2]),(x[3]),(x[4]),(x[5]),(x[6]),(x[7]),(x[8]),(x[9]),(x[10]),(x[11])))


In [ ]:
# total rows in data set after removing the header rows
dataset.count()

15060

In [ ]:
numPartitions = 2
support=80
#value of 80 is chosen because mam asked to pick 80% frequently cocusing patterns in each partition. Support of 80 value is converted into 80% in the Apriori function through a formula.

In [ ]:
#MapReduce Phase 1 for apriori algorithem
# In this phase we'll run apriori at each partition to find frequent items.
frequent_itemset = dataset.mapPartitions(apriori).groupByKey().map(lambda x:x[0]).collect()

In [ ]:
print(frequent_itemset)

['Young-Age', 'Private', ' Male', 'Part-Time-Hours', ' White', (' Male', 'Private'), ('Part-Time-Hours', 'Private'), (' White', 'Private'), (' White', ' Male'), (' White', 'Part-Time-Hours'), (' <=50K.', ' United-States'), ('HS-Graduates', ' United-States'), ('Married', ' United-States'), (' <=50K.', 'HS-Graduates'), ('Married', ' Husband'), (' Male', 'Private', ' United-States'), ('Part-Time-Hours', 'Private', ' United-States'), ('Part-Time-Hours', ' <=50K.', 'Private'), (' White', 'Private', ' United-States'), (' White', ' <=50K.', 'Private'), (' White', ' Male', ' United-States'), (' White', ' <=50K.', ' Male'), (' White', 'Part-Time-Hours', ' United-States'), (' White', ' <=50K.', 'Part-Time-Hours'), ('HS-Graduates', ' <=50K.', ' United-States'), ('Private', ' White', ' <=50K.', ' United-States'), ('HS-Graduates', ' <=50K.'), (' Male', 'Part-Time-Hours'), ('Private', 'Part-Time-Hours'), ('Private', 'Part-Time-Hours', ' United-States'), ' United-States', ' <=50K.', 'HS-Graduates', '

In [ ]:
len(frequent_itemset)

58

In [ ]:
#MapReduce Phase 2
# In this phase we'll count total occurance of each item set found in phase 1 in complete dataset
frequent_set_counts = dataset.mapPartitions(lambda x: totalCount_itemset(x,frequent_itemset)).reduceByKey(lambda x,y: x+y).collect()

In [ ]:
print(frequent_set_counts)

[((' Male', 'Private'), 7235), (('Part-Time-Hours', 'Private'), 7192), ((' White', 'Private'), 9492), ((' White', ' Male'), 8982), ((' White', 'Part-Time-Hours'), 7882), ((' <=50K.', ' United-States'), 10335), (('HS-Graduates', ' United-States'), 7667), (('Married', ' United-States'), 6379), ((' <=50K.', 'HS-Graduates'), 6711), (('Married', ' Husband'), 6203), ((' Male', 'Private', ' United-States'), 6544), (('Part-Time-Hours', 'Private', ' United-States'), 6429), (('Part-Time-Hours', ' <=50K.', 'Private'), 5948), ((' White', 'Private', ' United-States'), 8812), ((' White', ' <=50K.', 'Private'), 7319), ((' White', ' Male', ' United-States'), 8373), ((' White', ' <=50K.', ' Male'), 6098), ((' White', 'Part-Time-Hours', ' United-States'), 7288), ((' White', ' <=50K.', 'Part-Time-Hours'), 6222), (('HS-Graduates', ' <=50K.', ' United-States'), 6290), (('Private', ' White', ' <=50K.', ' United-States'), 6728), (('HS-Graduates', ' <=50K.'), 6711), ((' Male', 'Part-Time-Hours'), 6028), (('Pr

In [ ]:
len(frequent_set_counts)

58

In [ ]:
support = 0.4
frequent_items = []
# now based upon actual total occurance count of each itemset, we keep some best of them using support threshold
for item in frequent_set_counts:
    if item[1]/dataset.count() >= support:
        frequent_items.append(item)

In [ ]:
print(frequent_items)

[((' Male', 'Private'), 7235), (('Part-Time-Hours', 'Private'), 7192), ((' White', 'Private'), 9492), ((' White', ' Male'), 8982), ((' White', 'Part-Time-Hours'), 7882), ((' <=50K.', ' United-States'), 10335), (('HS-Graduates', ' United-States'), 7667), (('Married', ' United-States'), 6379), ((' <=50K.', 'HS-Graduates'), 6711), (('Married', ' Husband'), 6203), ((' Male', 'Private', ' United-States'), 6544), (('Part-Time-Hours', 'Private', ' United-States'), 6429), ((' White', 'Private', ' United-States'), 8812), ((' White', ' <=50K.', 'Private'), 7319), ((' White', ' Male', ' United-States'), 8373), ((' White', ' <=50K.', ' Male'), 6098), ((' White', 'Part-Time-Hours', ' United-States'), 7288), ((' White', ' <=50K.', 'Part-Time-Hours'), 6222), (('HS-Graduates', ' <=50K.', ' United-States'), 6290), (('Private', ' White', ' <=50K.', ' United-States'), 6728), (('HS-Graduates', ' <=50K.'), 6711), ((' Male', 'Part-Time-Hours'), 6028), (('Private', 'Part-Time-Hours'), 7192), (('Private', 'Pa

In [ ]:
len(frequent_items)

56

In [ ]:
temp=[]
# to generate rule, we save first item of each itemset in arr.
for item in frequent_items:
  # save all single value frequnet items in temp array.
  if (len(item[0])==1): 
      temp.append(item)
print(temp)

[((' United-States',), 13788), ((' <=50K.',), 11360), (('HS-Graduates',), 8164), (('Married',), 7001), ((' Husband',), 6203), (('Young-Age',), 6777), (('Private',), 11021), ((' Male',), 10147), (('Part-Time-Hours',), 9430), ((' White',), 12970)]


In [ ]:
N = dataset.count()

# loop through frequent item set
for cand in frequent_items:
  result=[]
  var1 = var2 = var3=0
  for t in temp:
    # finding confidence
    if ((cand[0])[0]==(t[0])[0]):     
      var1 = cand[1]/N
      var2 = t[1]/N
      confidence = var1/var2
    #  for frequent itemsets of size 2
    if ((len(cand[0]) == 2)):
      if ((cand[0])[1]==(t[0])[0]):
        var3 = t[1]/N
        if ((var2 == 0) | (var3 == 0)):
          lift = var1
        elif ((var2 != 0) & (var3 != 0)):
          lift = var1/(var2*var3)
          # for frequent candidate itemset of size greater than 2.
    else:
      if (var2 != 0):
        lift = var1/(var1*var2)

  result.append(cand[0]) 
  result.append(confidence)
  result.append(lift)
  sup=var1
  result.append(sup)
  
  if (((len(result[0])) == 2) & (confidence > 0.1)& (lift > 0.4)):
    print((result[0])[0]+" --> "+(result[0])[1]+ " "+" sup: "+str(result[3])+ " conf: "+str(result[1])+ " lift: " + str(result[2]))
    
  if (((len(result[0])) == 3) & (confidence > 0.1)& (lift > 0.4)):
    print((result[0])[0]+" , "+(result[0])[1]+" --> "+(result[0])[2]+ " "+" sup: "+str(result[3])+ " conf: "+ str(result[1])+ " lift: " + str(result[2]))
   

 <=50K. --> HS-Graduates  sup: 0.4456175298804781 conf: 0.5907570422535211 lift: 1.0897600509968186
Married -->  Husband  sup: 0.41188579017264276 conf: 0.8860162833880875 lift: 2.15112126839023
 Male , Private -->  United-States  sup: 0.4345285524568393 conf: 0.6449196806938011 lift: 1.4841825170000986
Part-Time-Hours , Private -->  United-States  sup: 0.42689243027888446 conf: 0.6817603393425239 lift: 1.5970307529162249
 White , Private -->  United-States  sup: 0.5851261620185922 conf: 0.6794140323824209 lift: 1.161141094834233
 White ,  <=50K. --> Private  sup: 0.4859893758300133 conf: 0.564302235929067 lift: 1.161141094834233
 White ,  Male -->  United-States  sup: 0.5559760956175299 conf: 0.6455666923670008 lift: 1.161141094834233
 White ,  <=50K. -->  Male  sup: 0.4049136786188579 conf: 0.47016191210485736 lift: 1.161141094834233
 White , Part-Time-Hours -->  United-States  sup: 0.4839309428950863 conf: 0.5619121048573631 lift: 1.161141094834233
 White ,  <=50K. --> Part-Time-Hou